In [2]:
import astropy
import matplotlib.pyplot as plt
import numpy as np
import scipy
from astropy.io import fits
import glob
import re

In [2]:
# prepare dark currents
n = '.005'

result = []
for fname in np.sort(glob.glob('./raw_dark/{}/*.fit'.format(n))):
    data = fits.open(fname)[0].data 
    header = fits.open(fname)[0].header
    result.append(data)
mean_data = np.median(result, axis=0) # mean or median
print(mean_data.shape)

file_name = './dark/0{}.fit'.format(n)
fits.writeto(filename=file_name, data=mean_data, header=header, overwrite=True)
f = fits.open(file_name)
print(f[0].data)
f.close()

(1336, 2004)
[[644.5 640.5 634.5 ... 572.  569.5 562.5]
 [640.  639.  641.  ... 571.5 567.5 569.5]
 [644.  643.  641.  ... 567.5 564.  566. ]
 ...
 [712.  714.  715.5 ... 647.  639.5 642. ]
 [713.5 704.  714.  ... 637.5 643.5 637. ]
 [717.5 710.  715.5 ... 643.  640.5 632. ]]


In [3]:
# master flat

result = []
for fname in np.sort(glob.glob('./Vflats-F2022/*.fts')):
    data = fits.open(fname)[0].data 
    header = fits.open(fname)[0].header
    result.append(data)
mean_data = np.median(result, axis=0) # mean or median
print(mean_data.shape)

file_name = './flat/master.fit'
fits.writeto(filename=file_name, data=mean_data, header=header, overwrite=True)
f = fits.open(file_name)
print(f[0].data)
f.close()

(1336, 2004)
[[7572.5 7649.  7634.5 ... 7595.  7632.5 7527. ]
 [7528.  7659.5 7673.  ... 7563.5 7599.5 7669. ]
 [7542.5 7675.5 7546.  ... 7653.5 7692.  7671. ]
 ...
 [8210.5 8314.5 8327.5 ... 8419.5 8416.5 8297. ]
 [8283.5 8241.5 8371.5 ... 8280.5 8275.  8400. ]
 [8302.5 8300.  8361.5 ... 8418.5 8399.  8271. ]]


In [ ]:
def fname_to_img(fname):
    '''
    must be valid fname
    '''
    return fits.open(fname)[0].data

In [ ]:
def reduction(img, texp=1):
    '''
    applies dark current and flat field corrections
    img = 2D pixel array
    texp = exposure time in seconds, valid options: [0.005,0.01,0.05,0.1,1,5]
    '''
    master_flat_field = fits.open('./flat/master.fit')[0].data
    dark_current = fits.open('./dark/{}.fit'.format(str(texp)))[0].data
    return (img - dark_current)/((master_flat_field-dark_current)/np.median(master_flat_field))

In [ ]:
#jupiter 1

day = 1
texp = 1
colors = ['green']
# colors = ['green', 'red', 'blue']


result = []
for c in colors:
    files = np.sort(glob.glob('./jupiter/1s/{}/*.fit'.format(c)))
    header = fits.open(files[0])[0].header
    mean_data = np.mean(np.array([reduction(fname_to_img(f), texp=texp) for f in files[:]]), axis=0)
    
    file_name = './data/{}_{}_{}.fit'.format(str(day),str(texp),c)
    fits.writeto(filename=file_name, data=mean_data, header=header, overwrite=True)
    f = fits.open(file_name)
    print(f[0].data)
    f.close()
result = np.array(result)

In [ ]:
# jupiter2

day = 2
texp = .005
colors = ['green', 'red', 'blue']


result = []
for c in colors:
    files = np.sort(glob.glob('./jupiter2/' + str(c[0]) + '_005/*.fit'))
    header = fits.open(files[0])[0].header
    mean_data = np.mean(np.array([reduction(fname_to_img(f), texp=texp) for f in files[:]]), axis=0)
    
    file_name = './data/{}_{}_{}.fit'.format(str(day),str(texp),c)
    fits.writeto(filename=file_name, data=mean_data, header=header, overwrite=True)
    f = fits.open(file_name)
    print(f[0].data)
    f.close()
result = np.array(result)

In [16]:
result = []
data_files = np.sort(glob.glob('./data/*.fit'))
for file in data_files:
    f = fits.open(file)
    data = f[0].data
    header = f[0].header

    adj_data = data.copy()
    adj_data[data < 0] = 0
    
    fits.writeto(filename=file, data=adj_data, header=header, overwrite=True)